# 중량제한

https://www.acmicpc.net/problem/1939

In [ ]:
%%writefile ex1.txt
3 3
1 2 2
3 1 3
2 3 2
1 3

In [ ]:
%%writefile ex1.txt
3 2
1 2 1
2 3 2
1 2

In [ ]:
%%writefile ex1.txt
10000 3
9998 9999 2
10000 9998 3
9999 10000 2
9998 10000

## 이진 탐색 + bfs 풀이
- 이진 탐색(Binary Search)을 사용하여 주어진 그래프에서 X부터 Y까지 경로를 찾을 때, 최대로 가능한 가중치를 구함

In [ ]:
from collections import deque
n, m = map(int, input().split())
adj = [[] for _ in range(n + 1)]
def bfs(c):
    queue = deque([start_node])
    visited = [False] * (n + 1)
    visited[start_node] = True
    while queue:
        x = queue.popleft()
        for y, weight in adj[x]:
            if not visited[y] and weight >= c:
                visited[y] = True
                queue.append(y)
    return visited[end_node]
start = 1000000000
end = 1
for _ in range(m):
    x, y, weight = map(int, input().split())
    adj[x].append((y, weight))
    adj[y].append((x, weight))
    start = min(start, weight)
    end = max(end, weight)
start_node, end_node = map(int, input().split())
result = start
while(start <= end):
    mid = (start + end) // 2 # mid는 현재의 중량을 의미합니다.
    if bfs(mid): # 이동이 가능하므로, 중량을 증가시킵니다.
        result = mid
        start = mid + 1
    else: # 이동이 불가능하므로, 중량을 감소시킵니다.
        end = mid - 1
print(result)

## 이진 탐색 + bfs 풀이 + 자료구조 개선
- 딕셔너리 안에 딕셔너리
    - 같은 경로가 여러개 있을 수 있어서, 중량이 제일 높은 경로만 냅둠

In [ ]:
from collections import deque

lines = open('ex1.txt').readlines()
N, M = map(int, lines[0].split())
graph = {i : {} for i in range(1, N + 1)}
X, Y = map(int, lines[-1].split())

def bfs(c):
    queue = deque([X])
    visited = [False] * (N + 1)
    visited[X] = True
    while queue and not visited[Y]:
        x = queue.popleft()
        for y, weight in graph[x].items():
            if not visited[y] and weight >= c:
                visited[y] = True
                queue.append(y)
    return visited[Y]

start = 100000
end = 1
for line in lines[1:-1]:
    x, y, weight = [*map(int, line.split())]
    if y not in graph[x].keys() or weight > graph[x][y]:
        graph[x][y] = weight
        graph[y][x] = weight
    start = min(start, weight)
    end = max(end, weight)
answer = start
while (start <= end):
    mid = (start + end) // 2
    if bfs(mid):
        answer = mid
        start = mid + 1
    else:
        end = mid - 1

print(answer)

### 가독성 개선
- 시간이 난다면...

In [ ]:
from collections import deque

def read_graph():
    lines = open('ex1.txt').readlines()
    N, M = map(int, lines[0].split())
    graph = {i : {} for i in range(1, N + 1)}
    X, Y = map(int, lines[-1].split())
    start = 100000
    end = 1
    for line in lines[1:-1]:
        x, y, weight = [*map(int, line.split())]
        if y not in graph[x].keys() or weight > graph[x][y]:
            graph[x][y] = weight
            graph[y][x] = weight
        start = min(start, weight)
        end = max(end, weight)
    return graph, X, Y, start, end

def bfs(graph, X, Y, c):
    queue = deque([X])
    visited = [False] * (len(graph) + 1)
    visited[X] = True
    while queue:
        x = queue.popleft()
        for y, weight in graph[x].items():
            if not visited[y] and weight >= c:
                visited[y] = True
                queue.append(y)
                if y == Y:
                    return True
    return visited[Y]

def binary_search(graph, X, Y, start, end):
    answer = start
    while (start <= end):
        mid = (start + end) // 2
        if bfs(graph, X, Y, mid):
            answer = mid
            start = mid + 1
        else:
            end = mid - 1
    return answer

if __name__ == "__main__":
    graph, X, Y, start, end = read_graph()
    answer = binary_search(graph, X, Y, start, end)
    print(answer)

## 알고리즘 개선
- [rlatndhrlatndh](https://www.acmicpc.net/source/53141888)
- Union-Find(Disjoint Set) 자료구조 / 알고리즘
    -  루트 노드를 찾음
- Kruskal 알고리즘 : 중량을 큰 기준으로 정렬함
    - X와 Y가 같은 집합에 속해있으면 최소 비용 신장 트리를 완성


In [ ]:
def find(x):
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]

def union(a,b):
    a = find(a)
    b = find(b)
    if a<b:
        parent[b] = a
    elif a>b:
        parent[a] = b

lines = open('ex1.txt').readlines()
N, M = map(int, lines[0].split())
parent = [i for i in range(N + 1)]
graph = []
ans = []
for line in lines[1:-1]:
    x, y, c = [*map(int, line.split())]
    graph.append((x,y,c))
graph=sorted(graph, key = lambda x:-x[2])
X, Y = map(int, lines[-1].split())
for x,y,c in graph:
    if find(x) == find(y):
        continue
    union(x, y)
    ans.append(c)
    if find(X) == find(Y):
        break
if ans:
    print(min(ans))
else:
    print(0)
